In [1]:
import pandas as pd

import findlay2025a as f25a
import wisc_ecephys_tools as wet

/home/gfindlay/projects/ece/gfys_workspace/.venv/lib/python3.13/site-packages/umap/__init__.py:9: ImportWarning: Tensorflow not installed; ParametricUMAP will be unavailable
  warn(


In [2]:
nbsh = wet.get_sglx_project("seahorse")
experiment = wet.rats.constants.SleepDeprivationExperiments.NOD
sortings = f25a.units.get_nod_sortings()

In [3]:
df = pd.read_parquet(nbsh.get_project_file("aggregated_cell_metrics.pqt"))

In [4]:
id_cols = ["subject", "experiment", "probe", "cluster_id"]


def copy_cell_types_to_other_states(
    df: pd.DataFrame, src_state: str, cell_type_col: str
) -> pd.DataFrame:
    state_cell_types = df[df["state"] == src_state][id_cols + [cell_type_col]].dropna()
    for _, row in state_cell_types.iterrows():
        # Find all rows with the same cell_id_cols
        mask = (df[id_cols] == row[id_cols]).all(axis=1)
        # Update cell_type_col for all states of this cell
        df.loc[mask, cell_type_col] = row[cell_type_col]
    return df


df.drop(
    columns=["cluster_cell_type", "narrow_wide_cell_type", "petersen_cell_type"],
    inplace=True,
    errors="ignore",
)

In [5]:
def classify_narrow_wide(peak_to_valley: float) -> str:
    if peak_to_valley <= 0.000425:
        return "narrow"
    else:
        return "wide"


narrow_wide_cell_types = (
    df.loc[
        (df["state"] == "NREM")
        & ((df["region"] == "cortical") | (df["region"] == "hippocampal")),
        "peak_to_valley",
    ]
    .apply(classify_narrow_wide)
    .rename("narrow_wide_cell_type")
)
df.loc[narrow_wide_cell_types.index, "narrow_wide_cell_type"] = narrow_wide_cell_types
df = copy_cell_types_to_other_states(df, "NREM", "narrow_wide_cell_type")

In [6]:
def classify_petersen_cortical_cell_type(peak_to_valley: float, tau_rise: float) -> str:
    if classify_narrow_wide(peak_to_valley) == "narrow":
        return "narrow interneuron"
    elif tau_rise > 6:
        return "wide interneuron"
    else:
        return "pyramidal"


petersen_cell_types = (
    df.loc[
        (df["state"] == "NREM") & (df["region"] == "cortical"),
        ["peak_to_valley", "tau_rise"],
    ]
    .apply(
        lambda row: classify_petersen_cortical_cell_type(
            row["peak_to_valley"], row["tau_rise"]
        ),
        axis=1,
    )
    .rename("petersen_cell_type")
)
df.loc[petersen_cell_types.index, "petersen_cell_type"] = petersen_cell_types
df = copy_cell_types_to_other_states(df, "NREM", "petersen_cell_type")


In [7]:
def classify_petersen_hippocampal_cell_type(
    peak_to_valley: float, tau_rise: float
) -> str:
    if classify_narrow_wide(peak_to_valley) == "narrow":
        return "narrow interneuron"
    elif tau_rise > 3:
        return "wide interneuron"
    else:
        return "pyramidal"


petersen_cell_types = (
    df.loc[
        (df["state"] == "NREM") & (df["region"] == "hippocampal"),
        ["peak_to_valley", "tau_rise"],
    ]
    .apply(
        lambda row: classify_petersen_hippocampal_cell_type(
            row["peak_to_valley"], row["tau_rise"]
        ),
        axis=1,
    )
    .rename("petersen_cell_type")
)
df.loc[petersen_cell_types.index, "petersen_cell_type"] = petersen_cell_types
df = copy_cell_types_to_other_states(df, "NREM", "petersen_cell_type")

In [8]:
cell_types = df.loc[
    df["state"] == "NREM", id_cols + ["narrow_wide_cell_type", "petersen_cell_type"]
].reset_index(drop=True)
cell_types.to_parquet(nbsh.get_project_file("cell_types.pqt"))

In [9]:
df.to_parquet(nbsh.get_project_file("aggregated_cell_metrics.pqt"))